<a href="https://colab.research.google.com/github/centeno/covid-impact-scrapper/blob/master/step01_collection_data_atlasMongoDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## SEMPRE RODAR ANTES

### Instalação das dependencias

In [1]:
#!apt-get update # to update ubuntu to correctly run apt install

# scraper
#!pip install selenium
#!apt install chromium-chromedriver
#!pip install webdriver-manager
#!cp /usr/lib/chromium-browser/chromedriver/usr/bin

!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium msticpy tld folium
!pip pymongo pymongo[srv] dnspython

# geocoder ipaddress
#!pip install msticpy

# get domain
#!pip3 install tld


# spacy
!pip install -U pip setuptools wheel
!pip install -U spacy
#!python -m spacy download en
#!python -m spacy download pt
!python -m spacy download en_core_web_sm
# spacy contextual Spell Check


!pip install contextualSpellCheck
!curl https://raw.githubusercontent.com/codelucas/newspaper/master/download_corpora.py | python3


Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Pa

ERROR: unknown command "pymongo"
     |████████████████████████████████| 1.6MB 5.1MB/s 
Requirement already up-to-date: setuptools in /usr/local/lib/python3.7/dist-packages (57.0.0)
Requirement already up-to-date: wheel in /usr/local/lib/python3.7/dist-packages (0.36.2)
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 12.8 MB 70 kB/s 
     |████████████████████████████████| 42 kB 1.0 MB/s 
     |████████████████████████████████| 9.1 MB 40.8 MB/s 
     |████████████████████████████████| 618 kB 40.1 MB/s 
     |████████████████████████████████| 456 kB 45.0 MB/s 
     |████████████████████████████████| 113 kB 62.1 MB/s 
  Created wheel for smart-open: filename=smart_open-3.0.0-py3-none-any.whl size=107106 sha256=7952a34a86103d968425e16ad8100600dd94239cbb09a6d47e5ea230343d056f
  Stored in directory: /root/.cache/pip/wheels/83/a6/12/bf3c1a667bde4251be5b7a3368b2d604c9af2105b5c1cb1870
Succe

### Import das dependencias

In [2]:
#from kora.selenium import wd as driver

import os
import selenium
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
#from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, date, timedelta

import urllib
import socket
from msticpy.sectools.geoip import IPStackLookup

import spacy
import contextualSpellCheck

import pymongo
import pandas as pd
from datetime import datetime as dt
from datetime import date, timedelta
from pandas.tseries.offsets import DateOffset
import dns

import tld
from tld import get_tld, get_fld

from google.colab import drive

### Connect google drive

In [3]:
# Codigo para montar o Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/My Drive/covid19impact/data/2021/01_collect/en

/content/drive/My Drive/covid19impact/data/2021/01_collect/en


In [5]:
# Listar os arquivos da pasta do projeto no Google Drive
os.chdir("/content/drive/My Drive/covid19impact/data/2021/01_collect/en/")
#!ls

### Import chaves de api

In [6]:
#home
keys = pd.read_csv("/content/drive/My Drive/covid19impact/keys/credentials.csv", sep=';')
access_token_ipstack = keys.loc[0,"key"]
access_token_pymongo = keys.loc[1,"key"]

### Criação das funções

In [7]:
import pandas as pd
GSE_RESULTS_CLASS = 'ZINbbc xpd O9g5cc uUPGi' # 'g' # 'tF2Cxc' # rc
GSE_MAX_RESULTS = 200

NLP = spacy.load("en_core_web_sm")
contextualSpellCheck.add_to_pipe(NLP)

def get_driver():
  options = webdriver.ChromeOptions()
  options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  options.add_argument('user-agent=Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1667.0 Safari/537.36')    
  return webdriver.Chrome('chromedriver',options=options)

def page_source(query, date):
  d = date.strftime('%Y%m%d')
  q = "{} after:{} and before:{}".format(query, date.strftime('%Y-%m-%d'), date.strftime('%Y-%m-%d'))
  url = "https://www.google.com/search?q={}&sort=date:r:{}:{}&num={}".format(q, d, d, GSE_MAX_RESULTS)
  driver = get_driver()
  driver.get(url)
  soup = BeautifulSoup(driver.page_source, 'html.parser') 
  return driver.page_source
  
def fetch_result(date, item):
  try:
    return {
        'date': date.strftime('%Y-%m-%d'),
        'header' : item.select_one('h3').text,
        'summary': item.findChildren('div', recursive=False)[-1].text,
        'link' : item.find('a', href=True)['href'].replace('/url?q=', ''),
    }
  except:
    return None

def search(query, date):
  soup = BeautifulSoup(page_source(query, date), 'html.parser') 
  html_results = soup.find_all('div', {'class': GSE_RESULTS_CLASS })
  results = [fetch_result(date, r) for r in html_results]
  return pd.DataFrame(list(filter(None, results)))

def fetch_domain(df):
  df['protocol'],df['domain'],df['path'],df['query'],df['fragment'] = zip(*[urllib.parse.urlsplit(x) for x in df.link])
  return df

def fetch_hostname(df): 
  urls = df['link']
  hostnames = []
  for url in urls:
    try:
      res = get_tld(url, as_object=True)
      hostnames.append(res.fld)
    except:
      res = np.nan
      hostnames.append(res)
  df['host_name'] = hostnames
  return df

def fetch_ipaddress(df):
  domains = df['domain']
  ipaddress = []
  for i in domains:
      i_strip = i.strip()
      try:
          ip = socket.gethostbyname(i_strip)
          ipaddress.append(ip)
      except socket.gaierror:
        ipaddress.append('127.0.0.1')
  df['ip_address'] = ipaddress
  return df

def fetch_location(df):
  iplocation = IPStackLookup(api_key=access_token_ipstack)
  df = iplocation.df_lookup_ip(df, column="ip_address")
  return df

def fetch_nlp(df):
  df['header_context_true'] = df['header'].apply(lambda x: str(NLP(x)))
  df['summary_context_true'] = df['summary'].apply(lambda x: str(NLP(x)))
  return df

def save(df):
  client = pymongo.MongoClient(access_token_pymongo)
  db = client.covid_impact
  df = df.astype('str')
  db.impacts.insert_many(df.to_dict('records'))
  

def load():
  client = pymongo.MongoClient(access_token_pymongo)
  db = client.covid_impact
  df = pd.DataFrame.from_records(db.impacts.find())
  return df

### Merge csv's ou Cria novos dados se já não existir uma base criada

In [8]:
# print
df = load()
df.tail()

,_id,date,header,summary,link,header_context_true,summary_context_true,protocol,domain,path,query,fragment,host_name,ip_address,level_0,index,CountryCode,CountryName,State,City,Longitude,Latitude,Asn,edges,Type,AdditionalData,IpAddress
2047,60ce21afa2bdc21e28138605,2021-03-20,Handbook of Research on Software Engineering a...,Strategic issues in distributed development • ...,https://books.google.com/books?id=7O30Y57dNXgC...,Handbook of Research on Software Engineering a...,Strategic issues in distributed development • ...,https,books.google.com,/books,id=7O30Y57dNXgC&pg=PR26&lpg=PR26&dq=%22covid+i...,,google.com,142.250.65.78,NaN,NaN,US,United States,Virginia,Herndon,-77.38275909423828,38.98371887207031,15169.0,set(),geolocation,{},142.250.65.78
2048,60ce21afa2bdc21e28138606,2021-03-20,Handbook of Research on Software Engineering a...,Strategic issues in distributed development • ...,https://books.google.com/books?id=7O30Y57dNXgC...,Handbook of Research on Software Engineering a...,Strategic issues in distributed development • ...,https,books.google.com,/books,id=7O30Y57dNXgC&pg=PR26&lpg=PR26&dq=%22covid+i...,,google.com,142.250.65.78,NaN,NaN,US,United States,Virginia,Herndon,-77.38275909423828,38.98371887207031,15169.0,set(),geolocation,{},142.250.65.78
2049,60ce21afa2bdc21e28138607,2021-03-20,"Innovation, Global Change and Territorial Resi...",... skill development Software Software develo...,https://books.google.com/books?id=mD2KL84p2QAC...,"Innovation, Global Change and Territorial Resi...",... skill development Software Software develo...,https,books.google.com,/books,id=mD2KL84p2QAC&pg=PA86&lpg=PA86&dq=%22covid+i...,,google.com,142.250.65.78,NaN,NaN,US,United States,Virginia,Herndon,-77.38275909423828,38.98371887207031,15169.0,set(),geolocation,{},142.250.65.78
2050,60ce21afa2bdc21e28138608,2021-03-20,"Innovation, Global Change and Territorial Resi...",... skill development Software Software develo...,https://books.google.com/books?id=mD2KL84p2QAC...,"Innovation, Global Change and Territorial Resi...",... skill development Software Software develo...,https,books.google.com,/books,id=mD2KL84p2QAC&pg=PA86&lpg=PA86&dq=%22covid+i...,,google.com,142.250.65.78,NaN,NaN,US,United States,Virginia,Herndon,-77.38275909423828,38.98371887207031,15169.0,set(),geolocation,{},142.250.65.78
2051,60ce21afa2bdc21e28138609,2021-03-20,"Innovation, Global Change and Territorial Resi...",... skill development Software Software develo...,https://books.google.com/books?id=mD2KL84p2QAC...,"Innovation, Global Change and Territorial Resi...",... skill development Software Software develo...,https,books.google.com,/books,id=mD2KL84p2QAC&pg=PA86&lpg=PA86&dq=%22covid+i...,,google.com,142.250.65.78,NaN,NaN,US,United States,Virginia,Herndon,-77.38275909423828,38.98371887207031,15169.0,set(),geolocation,{},142.250.65.78


In [9]:
len_files = df.shape[0]
len_files

2052

In [10]:
# get last date from table
len_files = len(df)

if len_files >= 1:
  df['date'] = pd.to_datetime(df['date'], dayfirst = False)
  last_date_table = df['date'].max()
  next_date = (last_date_table.date() + DateOffset(days = 1)).date()
  print('Last date is: ' + str(last_date_table.date()))
  print('Next date is: ' + str(next_date))
  print('\n')
  print('Shape: ' + str(df.shape))
else:
    # get first date
    next_date = date(2019, 9, 30)
    print(next_date)

Last date is: 2021-03-20
Next date is: 2021-03-21


Shape: (2052, 27)


## SCRAPER

In [11]:
keyword = '"covid impact" OR "disaster" OR "resilience" AND "software engineers" OR "software development" OR "disaster informatics" AND "productivity" OR "work from home" OR "remote working"'

if len_files >= 1:
  date = next_date 
else:
  date = datetime(2019, 9, 30)

In [12]:
#get seach
df_new = search(keyword, date)

#get domain
df_new = fetch_domain(df_new)

#get hostname
df_new = fetch_hostname(df_new)

#get ipaddress
df_new = fetch_ipaddress(df_new)

#get geocoded ipaddress
df_new = fetch_location(df_new)

In [13]:
#get nlp contextualSpellCheck 
df_new = fetch_nlp(df_new)

In [14]:
#save df new
save(df_new)

In [15]:
#print
df_new.head()

,date,header,summary,link,protocol,domain,path,query,fragment,host_name,ip_address,CountryCode,CountryName,State,City,Longitude,Latitude,Asn,edges,Type,AdditionalData,IpAddress,header_context_true,summary_context_true
0,2021-03-21,Pandemic programming | SpringerLink,"Sep 14, 2020 · To improve employee productivit...",https://link.springer.com/article/10.1007/s106...,https,link.springer.com,/article/10.1007/s10664-020-09875-y&sa=U&ved=2...,,,springer.com,151.101.0.95,US,United States,California,San Francisco,-122.392860,37.767849,54113.0,{},geolocation,{},151.101.0.95,Pandemic programming | SpringerLink,"Sep 14, 2020 · To improve employee productivit..."
1,2021-03-21,(PDF) Pandemic programming: How COVID-19 affec...,"Apr 29, 2021 · Conclusions: To improve employe...",https://www.researchgate.net/publication/34434...,https,www.researchgate.net,/publication/344342621_Pandemic_programming_Ho...,,,researchgate.net,104.17.33.105,US,United States,California,San Jose,-121.838226,37.330528,13335.0,{},geolocation,{},104.17.33.105,(PDF) Pandemic programming: How COVID-19 affec...,"Apr 29, 2021 · Conclusions: To improve employe..."
2,2021-03-21,How has the COVID-19 pandemic affected working...,"Mar 29, 2021 · Participants reported both impr...",https://figshare.manchester.ac.uk/articles/pre...,https,figshare.manchester.ac.uk,/articles/preprint/How_has_the_COVID-19_pandem...,,,manchester.ac.uk,18.200.90.253,IE,Ireland,Leinster,Dublin,-6.243330,53.353889,16509.0,{},geolocation,{},18.200.90.253,How has the COVID-19 pandemic affected working...,"Mar 29, 2021 · Participants reported both impr..."
3,2021-03-21,Pandemic programming: How COVID-19 affects sof...,"Sep 14, 2020 · To improve employee productivit...",https://pubmed.ncbi.nlm.nih.gov/32952438/&sa=U...,https,pubmed.ncbi.nlm.nih.gov,/32952438/&sa=U&ved=2ahUKEwit5LTEu6TxAhUFHrkGH...,,,nih.gov,34.107.134.59,US,United States,Missouri,Kansas City,-94.539612,39.107712,15169.0,{},geolocation,{},34.107.134.59,Pandemic programming: How COVID-19 affects sof...,"Sep 14, 2020 · To improve employee productivit..."
4,2021-03-21,New ASG Report Looks at COVID-19's Impact on S...,"Sep 22, 2020 · The Future of Remote Work and S...",https://devops.com/the-future-of-remote-work-a...,https,devops.com,/the-future-of-remote-work-and-software-develo...,,,devops.com,141.193.213.20,US,United States,Texas,Austin,-97.742310,30.267200,209242.0,{},geolocation,{},141.193.213.20,New ASG Report Looks at COVID-19's Impact on S...,"Sep 22, 2020 · The Future of Remote Work and S..."


In [16]:
# print atlas database
df = load()
df.sort_values(by=['date']).head(2)

,_id,date,header,summary,link,header_context_true,summary_context_true,protocol,domain,path,query,fragment,host_name,ip_address,level_0,index,CountryCode,CountryName,State,City,Longitude,Latitude,Asn,edges,Type,AdditionalData,IpAddress
149,605b283fdf20fc115ee7b04e,2019-10-01,6 charts to understand the coronavirus impact ...,"Apr 15, 2020 · COVID-19 is changing tech execu...",https://www.ciodive.com/news/coronavirus-covid...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,605b283fdf20fc115ee7b058,2019-10-01,The COVID-19 World And Its Impact On The Futur...,A Pandemic World – First Observations · The vi...,https://www.workdesign.com/2020/03/the-covid-1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# print last input
df.tail(2)

,_id,date,header,summary,link,header_context_true,summary_context_true,protocol,domain,path,query,fragment,host_name,ip_address,level_0,index,CountryCode,CountryName,State,City,Longitude,Latitude,Asn,edges,Type,AdditionalData,IpAddress
2164,60ce4a791a3f9eae0d4a6b52,2021-03-21,The Art of Lean Software Development: A Practi...,"If these were the only two trends, we would be...",https://books.google.com/books?id=0VsK9cVZauQC...,The Art of Lean Software Development: A Practi...,"If these were the only two trends, we would be...",https,books.google.com,/books,id=0VsK9cVZauQC&pg=PA2&lpg=PA2&dq=%22covid+imp...,,google.com,172.253.123.139,NaN,NaN,US,United States,California,Mountain View,-122.07540893554688,37.419158935546875,15169.0,set(),geolocation,{},172.253.123.139
2165,60ce4a791a3f9eae0d4a6b53,2021-03-21,The Art of Lean Software Development: A Practi...,"If these were the only two trends, we would be...",https://books.google.com/books?id=0VsK9cVZauQC...,The Art of Lean Software Development: A Practi...,"If these were the only two trends, we would be...",https,books.google.com,/books,id=0VsK9cVZauQC&pg=PA2&lpg=PA2&dq=%22covid+imp...,,google.com,172.253.123.139,NaN,NaN,US,United States,California,Mountain View,-122.07540893554688,37.419158935546875,15169.0,set(),geolocation,{},172.253.123.139
